In [ ]:
from illufly.llm import Memory, LiteLLM
from illufly.rocksdb import IndexedRocksDB

db = IndexedRocksDB("__DB2__")

In [7]:
llm = LiteLLM(model="qwen-plus", imitator="QWEN")
memory = Memory(llm, memory_db=db)

In [19]:
await memory.extract([
    {"role": "user", 
     "content": """
| 主题           | 问题                     | 答案                  |
|---------------|-------------------------|----------------------|
| [计算]比较策略 | 用户对数字比较的要求是？ | 按小数点后逐位比较「记住这个计算策略」 |
| [UI]按钮颜色  | 用户偏好什么色调？       | 蓝色系，透明度≤50%     |
"""}])

2025-04-01 11:10:17 - chromadb.segment.impl.vector.local_hnsw - WARNING - Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2



memory.retrieve >>>  |[计算]比较策略|用户对数字比较的要求是？|按小数点后逐位比较「记住这个计算策略」|

memory.extract >>> No tables extract


In [18]:
db.values()

[MemoryQA(user_id='default', topic='[计算]比较策略', question_hash='f28e2cd498d0fe64f8a1811e7645433e75e68a45bd736e41796623ccaadee2c3', question='用户对数字比较的要求是？', answer='按小数点后逐位比较「记住这个计算策略」', created_at=1743476116.065413)]

In [17]:
input_messages = [{
    "role": "user",
    "content": "你能帮我设计一个UI吗"
}]
CHROMA_COLLECTION = "memory"
results = await memory.retriver.query(
            texts=[memory.from_messages_to_text(input_messages)],
            user_id="default",
            collection_name=CHROMA_COLLECTION,
            threshold=2,
            query_config={"n_results": 10}
        )
print(results)
items = [f'|{r["topic"]}|{r["question"]}|{r["answer"]}|' for r in results[0]["metadatas"]]
uniq_items = "\n".join(list(dict.fromkeys(items)))
print("\nmemory.retrieve >>> ", uniq_items)


2025-04-01 11:05:32 - chromadb.segment.impl.vector.local_hnsw - WARNING - Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2


[{'text': 'user: 你能帮我设计一个UI吗', 'ids': ['cead50c666178048863872bc74737520', '1eba2cc4181e5944e967010f7f167067'], 'metadatas': [{'answer': '按小数点后逐位比较「记住这个计算策略」', 'created_at': 1743476116.065413, 'question': '用户对数字比较的要求是？', 'topic': '[计算]比较策略', 'user_id': 'default'}, {'answer': '按小数点后逐位比较「记住这个计算策略」', 'created_at': 1743476116.065413, 'question': '用户对数字比较的要求是？', 'topic': '[计算]比较策略', 'user_id': 'default'}], 'documents': ['用户对数字比较的要求是？', '按小数点后逐位比较「记住这个计算策略」'], 'distances': [0.7199002504348755, 0.7561813592910767]}]

memory.retrieve >>>  |[计算]比较策略|用户对数字比较的要求是？|按小数点后逐位比较「记住这个计算策略」|


In [9]:
import pandas as pd
import re
import html

def safe_extract_markdown_tables(md_text):
    """安全提取Markdown表格为结构化数据（支持多表）"""
    tables = []
    # 匹配所有表格（非贪婪模式）
    table_blocks = re.finditer(
        r'\|(.+?)\|\n\|([\-: ]+\|)+\n((?:\|.*\|\n?)+)',
        md_text,
        re.DOTALL
    )
    
    for table in table_blocks:
        try:
            headers = [h.strip() for h in table.group(1).split('|') if h.strip()]
            rows = []
            
            for row in table.group(3).split('\n'):
                if not row.strip():
                    continue
                cells = [
                    html.unescape(cell).strip() 
                    for cell in row.split('|')[1:-1]
                ]
                if len(cells) == len(headers):
                    rows.append(cells)
            
            if headers and rows:
                tables.append(pd.DataFrame(rows, columns=headers))
        except Exception as e:
            print(f"表格解析失败: {e}")
    
    return tables

# 测试用例
md_table = """
```markdown
| 主题           | 问题                     | 答案                  |
|---------------|-------------------------|----------------------|
| [计算]比较策略 | 用户对数字比较的要求是？ | 按小数点后逐位比较「记住这个计算策略」 |
| [UI]按钮颜色  | 用户偏好什么色调？       | 蓝色系，透明度≤50%     |
```
"""

tables = safe_extract_markdown_tables(md_table)
tables
# # 使用示例
# tables = safe_extract_markdown_tables(md_table)
# for df in tables:
#     print(df.to_dict('records'))

[         主题            问题                   答案
 0  [计算]比较策略  用户对数字比较的要求是？  按小数点后逐位比较「记住这个计算策略」
 1  [UI]按钮颜色     用户偏好什么色调？          蓝色系，透明度≤50%]

In [14]:
tables[0].to_dict()

{'主题': {0: '[计算]比较策略', 1: '[UI]按钮颜色'},
 '问题': {0: '用户对数字比较的要求是？', 1: '用户偏好什么色调？'},
 '答案': {0: '按小数点后逐位比较「记住这个计算策略」', 1: '蓝色系，透明度≤50%'}}